In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
# Import required modules
import re
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.oauth2.credentials import Credentials
from google.colab import auth

In [ ]:
# Authenticate the Colab environment
auth.authenticate_user()

In [ ]:
# Define the API scope
SCOPES = ['https://www.googleapis.com/auth/documents']

In [ ]:
# Build the Google Docs API service
from google.auth.transport.requests import Request
from google.auth import default

In [ ]:
creds, _ = default(scopes=SCOPES)
docs_service = build('docs', 'v1', credentials=creds)

In [ ]:
def markdown_to_google_docs_requests(markdown_text):
    import re

    requests = []
    current_index = 1  # Initialize startIndex to 1 (Google Docs indices are 1-based)

    lines = markdown_text.split("\n")
    for line in lines:
        line = line.strip()

        # Heading 1: Main title
        if line.startswith("# "):
            text = f"{line[2:]}\n\n"
            requests.append({'insertText': {'location': {'index': current_index}, 'text': text}})
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': current_index, 'endIndex': current_index + len(text)},
                    'paragraphStyle': {'namedStyleType': 'HEADING_1'},
                    'fields': 'namedStyleType'
                }
            })
            current_index += len(text)

        # Heading 2: Section headers
        elif line.startswith("## "):
            text = f"{line[3:]}\n\n"
            requests.append({'insertText': {'location': {'index': current_index}, 'text': text}})
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': current_index, 'endIndex': current_index + len(text)},
                    'paragraphStyle': {'namedStyleType': 'HEADING_2'},
                    'fields': 'namedStyleType'
                }
            })
            current_index += len(text)

        # Heading 3: Sub-section headers
        elif line.startswith("### "):
            text = f"{line[4:]}\n\n"
            requests.append({'insertText': {'location': {'index': current_index}, 'text': text}})
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': current_index, 'endIndex': current_index + len(text)},
                    'paragraphStyle': {'namedStyleType': 'HEADING_3'},
                    'fields': 'namedStyleType'
                }
            })
            current_index += len(text)

        # Checkbox items
        elif line.startswith("- [ ]"):
            text = f"☐ {line[5:]}\n"
            requests.append({'insertText': {'location': {'index': current_index}, 'text': text}})
            current_index += len(text)

        # Bullet points
        elif line.startswith("*") or line.startswith("-"):
            text = f"• {line[1:].strip()}\n"
            requests.append({'insertText': {'location': {'index': current_index}, 'text': text}})
            current_index += len(text)

        # Regular text
        elif line:
            text = f"{line}\n"
            requests.append({'insertText': {'location': {'index': current_index}, 'text': text}})
            current_index += len(text)

        # Blank lines
        else:
            text = "\n"
            requests.append({'insertText': {'location': {'index': current_index}, 'text': text}})
            current_index += len(text)

    return requests

In [ ]:

# Markdown meeting notes as a string
markdown_notes = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

In [ ]:
# Convert markdown to Google Docs API requests
requests = markdown_to_google_docs_requests(markdown_notes)

In [ ]:
# Create a new Google Doc
doc = docs_service.documents().create(body={'title': 'Product Team Sync Notes'}).execute()

In [ ]:
# Get the document ID
doc_id = doc.get('documentId')

In [ ]:
# Send the formatting requests to the Google Docs API
docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

{'replies': [{},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {}],
 'writeControl': {'requiredRevisionId': 'ALBJ4LutdBKfQ45BYR-kGwXrzH-O_7ADfb8fzsyRE5jCtNG64QfGjXpwcLd6Z60bgTU890dyhhNnOHzoOPXWjw'},
 'documentId': '1MozdfM2EghntQmjQFMA3A4jrUvKQijHbwgV_HalmOkw'}

In [ ]:
print(f"Document created successfully: https://docs.google.com/document/d/{doc_id}/edit")

Document created successfully: https://docs.google.com/document/d/1MozdfM2EghntQmjQFMA3A4jrUvKQijHbwgV_HalmOkw/edit
